In [2]:
import numpy as np
import pyvista as pv

import geogen.model as geo
import geogen.generation as gen
import geogen.plot as geovis
from geogen.probability import SedimentBuilder


pv.set_jupyter_backend('static')
WS = (600,400) # Set a custom window size to be reused in most plots


## GeoProcess Icon Sketchbook

Source code for generating GeoProcess sample images for documentation.

### Custom Plot Type for GeoProcess Icon Sketchbook 

In [3]:
def demoview(model, threshold=-0.5, opacity=1, plotter=None):

    if plotter is None:
        plotter = pv.Plotter(window_size=WS)
    p=plotter
    
    cats = np.arange(11)   # Set the number of categories for the colormap
    clim = [cats.min(), cats.max()] # Preset color limits for all subplots
    
    grid = geovis.get_voxel_grid_from_model(model, threshold=threshold)
    skin=grid.extract_surface()
    
    p.add_mesh(skin,scalars='values',clim=clim,cmap='gist_ncar',opacity=opacity)
    # p.add_axes(line_width=5)    
    p.remove_scalar_bar()
    
    flat_bounds = [item for sublist in model.bounds for item in sublist]
    bounding_box = pv.Box(flat_bounds)
    p.add_mesh(bounding_box, color="black", style="wireframe", line_width=3, name="bounding_box")
    p.camera.elevation = -20
    return p

def get_trimmed_bounding_box():
    new_points = np.array([
    [-20, -20, -10],
    [ 20, -20, -10],
    [-20,  20, -10],
    [ 20,  20, -10],
    [-20, -20,  10],
    [ 20, -20,  10],
    [-20,  20,  10],
])
    lines = [
        2, 0, 1,  # Line from point 0 to point 1
        2, 0, 2,  # Line from point 0 to point 2
        2, 0, 4,  # Line from point 0 to point 4
        2, 1, 3,  # Line from point 1 to point 3
        2, 1, 5,  # Line from point 1 to point 5
        2, 2, 3,  # Line from point 2 to point 3
        2, 2, 6,  # Line from point 2 to point 6
        2, 4, 5,  # Line from point 4 to point 5
        2, 4, 6   # Line from point 4 to point 6
    ]
    # Create a new PolyData for the bounding box
    line_polydata = pv.PolyData()
    line_polydata.points = new_points
    line_polydata.lines = lines
    return line_polydata

SCALING = 2.0


### Layer Demo

In [4]:
bounds = ((-20, 20), (-20, 20), (-10, 10))
resolution = (128, 128, 64)
model = geo.GeoModel(bounds=bounds, resolution=resolution)

In [5]:
layer = geo.Layer(base=-10, width=10, value=0)
model.clear_history()
model.add_history(layer)
model.compute_model()
p = demoview(model)
p.screenshot("images/layer1.png", scale=SCALING, transparent_background=True)

layer = geo.Layer(base=0, width=5, value=2)
model.clear_history()
model.add_history(layer)
model.compute_model()
p = demoview(model, opacity=.8, plotter=p)

p.screenshot("images/layer2.png", scale=SCALING, transparent_background=True)
print('')

### Shift Demo


In [5]:
bedrock = geo.Bedrock(base=-10, value=0)
sb = SedimentBuilder(1,10,4,4)
vals, thicks = sb.build_layers()
sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
fold = geo.Fold(strike=30, dip=90, rake=0, origin=(0, 0, 0), amplitude=4, period=90, shape=1)
dike = geo.DikePlane(strike=45, dip=90, width=4, origin=(0, 0, 0), value=6)

model.clear_history()
model.add_history([bedrock,sediment, fold, dike])
model.compute_model()
p = demoview(model)

p.screenshot("images/shift1.png", scale=SCALING, transparent_background=True)

shift = geo.Shift(vector=(8,-4,8))
model.add_history(shift)
model.compute_model()
p = demoview(model)

p.screenshot("images/shift2.png", scale=SCALING, transparent_background=True);

### Rotate Demo

In [6]:
def simple_hist():
    bedrock = geo.Bedrock(base=-10, value=0)
    sb = SedimentBuilder(1,10,4,4)
    vals, thicks = sb.build_layers()
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    fold = geo.Fold(strike=30, dip=90, rake=0, origin=(0, 0, 0), amplitude=4, period=90, shape=1)
    dike = geo.DikePlane(strike=45, dip=90, width=4, origin=(0, 0, 0), value=6)
    return [bedrock, sediment, fold, dike]


model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)

p.screenshot("images/rotate1.png", scale=SCALING, transparent_background=True)
axis = [0,0,1]
rot = geo.Rotate(axis=axis, angle=-30)
model.add_history(rot)
model.compute_model()
p = demoview(model)

p.screenshot("images/rotate2.png", scale=SCALING, transparent_background=True);

### Bedrock Demo

In [7]:
model.clear_history()
model.add_history(geo.NullProcess())
model.compute_model()
model.data[0] = 0 # Hack to bypass error warnings
p = demoview(model)
p.screenshot("images/bedrock1.png", scale=SCALING, transparent_background=True)

bedrock = geo.Bedrock(base=-2, value=0)
model.clear_history()
model.add_history(bedrock)
model.compute_model()
p = demoview(model)
p.screenshot("images/bedrock2.png", scale=SCALING, transparent_background=True);

### Sedimentation Demo

In [8]:
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/sediment1.png", scale=SCALING, transparent_background=True)

sediment2 = geo.Sedimentation(value_list=[5,9,7,8], thickness_list=[1,3,2,3])
model.add_history(sediment2)
model.compute_model()
p = demoview(model, plotter=p, opacity=0.7)
p.screenshot("images/sediment2.png", scale=SCALING, transparent_background=True);

### Unconformity Demo

In [9]:
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/unconf1.png", scale=SCALING, transparent_background=True)

unconformity = geo.UnconformityBase(-5)
model.add_history(unconformity)
model.compute_model()
p = demoview(model)
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model, plotter=p, opacity=0.2)
p.screenshot("images/unconfbase.png", scale=SCALING, transparent_background=True);


### Tilt Demo

In [10]:
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/tilt1.png", scale=SCALING, transparent_background=True)

tilt = geo.Tilt(strike=-45, dip=15, origin= (20,20,0))
model.add_history(tilt)
model.compute_model()
p = demoview(model)
p.screenshot("images/tilt2.png", scale=SCALING, transparent_background=True);

### Dike Intrusions Demo

In [11]:
name = "dikeplane"

def flat_hist():
    bedrock = geo.Bedrock(base=-5, value=0)
    sb = SedimentBuilder(1,10,4,4)
    vals, thicks = sb.build_layers()
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    fold = geo.Fold(strike=30, dip=90, rake=0, origin=(0, 0, 0), amplitude=4, period=90, shape=1)
    erode = geo.UnconformityDepth(4)
    return [bedrock, sediment, fold, erode]


model.clear_history()
model.add_history(flat_hist())
model.compute_model()
p = demoview(model,)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

dike = geo.DikePlane(strike=-60, dip=90, width=4, origin=(0, 0, 0), value=6)
model.add_history(dike)
model.compute_model()
p = demoview(model, )
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [12]:
name = "dikecolumn"

model.clear_history()
bedrock = geo.Bedrock(base=5, value=1)
model.add_history(bedrock)
model.compute_model()
p = demoview(model, opacity=.5)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

dikecol = geo.DikeColumn(origin=(0,0,6), diam = 30, value=6, minor_axis_scale=.6, rotation = -45, clip=True) 
model.add_history(dikecol)
model.compute_model()
model.data[model.data < 6] = -1
p = demoview(model, plotter=p)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [13]:
name = "dikehemisphere"

model.clear_history()
bedrock = geo.Bedrock(base=5, value=1)
model.add_history(bedrock)
model.compute_model()
p = demoview(model, opacity=.5)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

dikecol = geo.DikeHemisphere(origin=(-1,-1,-3), diam = 30, height=10, value=6, minor_axis_scale=.6, rotation = -45, clip=True) 
model.add_history(dikecol)
model.compute_model()
model.data[model.data < 6] = -1
p = demoview(model, plotter=p)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [14]:
def slice_volume(model, start, stop):
    # convert to theta coords
    theta = np.arctan2(model.xyz[:,1], model.xyz[:,0])
    mask = (theta >= start) & (theta <= stop)
    model.data[mask] = -1
    return model

In [15]:
name = "dikehemispherepushed"

def sed_hist():
    bedrock = geo.Bedrock(base=-5, value=0)
    sb = SedimentBuilder(1, 10, 4, 4)
    vals, thicks = sb.build_layers()
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    return [bedrock, sediment]

model.clear_history()
model.add_history(sed_hist())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=1)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

# Now, add your dike hemisphere and repeat for the second model state
dikecol = geo.DikeHemispherePushed(origin=(-1, -1, -5), diam=30, height=10, value=6, minor_axis_scale=0.6, rotation=-45, clip=True)
model.add_history(dikecol)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [16]:
name = "LaccolithLopolith"

model.clear_history()
model.add_history(sed_hist())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=.8)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

dikecol = geo.Laccolith(origin=(2,-1,0),cap_diam = 30, stem_diam = 5, height = 4, value=6 )
model.add_history(dikecol)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [17]:
name = "dikeplug"

model.clear_history()
bedrock = geo.Bedrock(base=5, value=1)
model.add_history(bedrock)
model.compute_model()
p = demoview(model, opacity=.5)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

dikecol = geo.DikePlug(origin=(0,0,5), diam = 30, value=6, minor_axis_scale=.4, rotation = -15, shape=10, clip=True) 
model.add_history(dikecol)
model.compute_model()
model.data[model.data < 6] = -1
p = demoview(model, plotter=p)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [18]:
name = "dikeplugpushed"

model.clear_history()
model.add_history(sed_hist())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=.8)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

dikecol = geo.DikePlugPushed(origin=(0,0,5), diam = 30, value=6, minor_axis_scale=.4, rotation = -15, shape=10) 
model.add_history(dikecol)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

### Tectonic Transforms

In [19]:
def cross_layers():
    bedrock = geo.Bedrock(base=-5, value=0)
    vals = [1,2,3,4,1,2,3,4]
    thicks = [1.5]
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    dikes = []
    for i in range(6):
        dike = geo.DikePlane(strike=0, dip=90, width=1, origin=(-16 + i*6, 0, 0), value=6)
        dikes.append(dike)
    return [bedrock, sediment] + dikes

In [20]:
name = "fold"

model.clear_history()
model.add_history(cross_layers())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=1)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

fold = geo.Fold(strike=45, dip=90, rake=45, origin=(0, 0, 0), amplitude=3, period=20, shape=.2)
model.add_history(fold)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [21]:
name = "fault"

model.clear_history()
model.add_history(cross_layers())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=1)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

fold = geo.Fault(strike=-90, dip=45, rake=45, origin=(0, 0, 0), amplitude=3)
model.add_history(fold)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [22]:
name = "shear"

model.clear_history()
model.add_history(cross_layers())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=1)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

fold = geo.Shear(strike=-90, dip=45, rake=45, origin=(0, 0, 0), amplitude=5, steepness=2)
model.add_history(fold)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [23]:
name = "slip"

model.clear_history()
model.add_history(cross_layers())
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, opacity=1)
p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

parabola = lambda x: -(.3*x)**2
fold = geo.Slip(displacement_func= parabola, strike=-90, dip=45, rake=45, origin=(0, 0, 0), amplitude=.4)
model.add_history(fold)
model.compute_model()
model = slice_volume(model, 0, 3*np.pi/4)
p = demoview(model, )

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);

In [38]:
name = "metaball"

model.clear_history()
model.add_history(geo.NullProcess())
model.compute_model()
model.data[0] = 0 # Hack to bypass error warnings
p = demoview(model)
p.screenshot(f"images/{name}1.png", scale=SCALING, transparent_background=True)

# Make a starting list of balls generatively and create a deterministic MetaBall object
ballgen = geo.BallListGenerator(step_range = [2,4], rad_range = [.5,1], goo_range=[.8,1])
balls = ballgen.generate(n_balls=52, origin=[0,0,-5], variance=2)

# Threshold 1 MetaBall
model.clear_history()
metaball = geo.MetaBall(balls=balls, threshold = 1, value = 6, clip=False)
model.add_history(metaball)
model.compute_model()
p = demoview(model, plotter=p)

p.remove_actor("bounding_box")
new_bound_box = p.add_mesh(get_trimmed_bounding_box(), color="black", line_width=3)
p.screenshot(f"images/{name}2.png", scale=SCALING, transparent_background=True);